In [ ]:
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import truncnorm

In [ ]:
def gen_cityblock_map(img_size: int, min_roads: int, max_roads: int, min_squares: int, max_squares: int, path: str, verbose=True):

  # image size
  big_img_size = int(img_size * 1.4142)

  # minimum and maximum number of skipped roads
  min_del_roads = int(0.1 * min_roads)
  max_del_roads = int(0.25 * max_roads)

  # minimum and maximum size of black squares
  min_square_size = 2
  max_square_size = 5

  image = Image.new("L", (big_img_size, big_img_size), color="black")
  draw = ImageDraw.Draw(image)

  # number of vertical and horizontal roads
  n_vertical = np.random.randint(min_roads, max_roads)
  n_horizontal = np.random.randint(min_roads, max_roads)

  # possible road widths
  line_widths = [3, 3, 6, 6, 6, 10]

  # shifts
  scale = 3.
  rng = 30
  shifts_v = truncnorm(a=-rng/scale, b=+rng/scale, scale=scale).rvs(size=n_vertical).round().astype(int)
  shifts_h = truncnorm(a=-rng/scale, b=+rng/scale, scale=scale).rvs(size=n_horizontal).round().astype(int)

  # vertical and horizontal lines
  v_del = np.random.randint(min_del_roads, max_del_roads)
  h_del = np.random.randint(min_del_roads, max_del_roads)
  vertical = np.delete(np.linspace(0, big_img_size, n_vertical) + shifts_v, np.random.randint(0, n_vertical, v_del))
  horizontal = np.delete(np.linspace(0, big_img_size, n_horizontal) + shifts_h, np.random.randint(0, n_horizontal, h_del))
  n_vertical = n_vertical - v_del 
  n_horizontal = n_horizontal - h_del

  # draw vertical roads
  for v in vertical:
    line_width = np.random.choice(line_widths)
    draw.line([(v, 0), (v, big_img_size)], fill="white", width=line_width)

  # draw horizontal roads
  for h in horizontal:
    line_width = np.random.choice(line_widths)
    draw.line([(0, h), (big_img_size, h)], fill="white", width=line_width)

  # draw black rectangles
  for _ in range(np.random.randint(min_squares, max_squares)):
    v_s = np.random.randint(min_square_size, max_square_size)
    h_s = np.random.randint(min_square_size, max_square_size)
    v_i = np.random.randint(0, n_vertical - max_square_size)
    h_i = np.random.randint(0, n_horizontal - max_square_size)

    v_1 = vertical[v_i]
    v_2 = vertical[v_i + v_s]
    h_1 = horizontal[h_i]
    h_2 = horizontal[h_i + h_s]

    draw.rectangle([(v_1+2, h_1+2), (v_2-2, h_2-2)], fill="black")

  # rotate img
  alpha = np.random.randint(0, 45)
  image = image.rotate(alpha)

  # crop img
  lt = (big_img_size - img_size) / 2
  rb = (big_img_size + img_size) / 2
  image = image.crop((lt, lt, rb, rb))

  if verbose:
    plt.imshow(image, cmap="gray")
    plt.axis("off")
    plt.show()
  image.save(path)

In [ ]:
np.random.seed(0)
img_size = 1024
min_roads = 12
max_roads = 16
min_squares = 3
max_squares = 7
path_base = "road"

for i in range(1000):
    path = f'{path_base}{str(i+1000).jpg}'
    gen_cityblock_map(img_size, min_roads, max_roads, min_squares, max_squares, path, False)